# Renforcement

In [1]:
### Import Lib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import CamembertTokenizer, CamembertForSequenceClassification
from transformers import AdamW
import locale
import dateparser
import torch
from torch import tensor
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import os
from tqdm.auto import tqdm

2024-01-30 20:40:33.795941: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv("../data/avis/df_cleaned.csv")
df.head()

,user,etoiles,n_avis,date_avis,titre_avis,text_avis,date_experience,Société,text_total
0,Mme Rakotondrasamy Lalanirina,1,3.0,2023-10-20 03:16:24+00:00,Orgueil gestionnaire,"Ça reste une mauvaise expérience, quand ton ge...",2023-10-19,MaBanqueBNPParibas,Orgueil gestionnaire Ça reste une mauvaise exp...
1,Florian T,1,2.0,2023-10-19 13:10:44+00:00,Mauvaise expérience en agence physique et en l...,"Mauvaise expérience en agence classique, mon c...",2023-10-19,MaBanqueBNPParibas,Mauvaise expérience en agence physique et en l...
2,Anca Visdei,5,1.0,2023-10-19 15:41:51+00:00,"Chère agence ,","Chère agence ,je dois bous dire que tant mon m...",2023-10-19,MaBanqueBNPParibas,"Chère agence , Chère agence ,je dois bous dire..."
3,Pat perso,1,5.0,2023-10-18 16:13:38+00:00,Mépris ou incompétence ou les 2,"RV pris à l'agence du Touquet, sur proposition...",2023-10-17,MaBanqueBNPParibas,Mépris ou incompétence ou les 2 RV pris à l'ag...
4,Sylvia FABRE,2,4.0,2023-10-17 22:22:10+00:00,Pas d'imprimé prévu pour transferts de compte ...,Pour une demande de transfert de fonds d'un co...,2023-10-17,MaBanqueBNPParibas,Pas d'imprimé prévu pour transferts de compte ...


In [ ]:
df.info()

In [4]:
df = df[df['Société'] != "YounitedCredit"]

"Mauvaise expérience en agence physique et en ligne Mauvaise expérience en agence classique, mon conseiller était plus dans la vente que dans le conseil.Je change alors pour la banque en ligne avec beaucoup d'espoir, la digitalisation ça a du bon.Eh bien c'est encore pire. Je n'ai plus 1 mais plusieurs conseillers et la qualité de service est au petit bonheur la chance.Pour une simple demande de modification de compte (purement administratif), ça fait 8 mois que j'attend. Quand je prend rendez-vous en ligne pour échanger sur l'avancement de ma demande, la conseillère ne se présente même pas.Fuyez.PS : je recommande Fortuneo ou Boursorama --> rapide, efficace et gratuit."

In [56]:
df = df.drop(["text_avis","titre_avis"], axis = 1)
df.head()

,user,etoiles,n_avis,date_avis,date_experience,Société,text_total,text_stop,text_lemma,label,score,sentiment_norm
0,Mme Rakotondrasamy Lalanirina,1,3.0,2023-10-20 03:16:24+00:00,2023-10-19,MaBanqueBNPParibas,orgueil gestionnaire ça reste une mauvaise exp...,"orgueil gestionnaire mauvaise expérience , ges...","orgueil gestionnaire mauvais expérience , gest...",-1,0.956365,-0.956365
1,Florian T,1,2.0,2023-10-19 13:10:44+00:00,2023-10-19,MaBanqueBNPParibas,mauvaise expérience en agence physique et en l...,mauvaise expérience agence physique ligne mauv...,mauvais expérience agence physique ligne mauva...,1,0.885947,0.885947
2,Anca Visdei,5,1.0,2023-10-19 15:41:51+00:00,2023-10-19,MaBanqueBNPParibas,chère agence chère agence je dois bous dire q...,"chère agence , chère agence , dois bous mari f...","cher agence , cher agence , devoir bou mari fi...",-1,0.976381,-0.976381
3,Pat perso,1,5.0,2023-10-18 16:13:38+00:00,2023-10-17,MaBanqueBNPParibas,mépris ou incompétence ou les 2 rv pris à l ag...,"mépris incompétence 2 rv pris agence touquet ,...",mépris incompétence 2 rv prendre agence touque...,-1,0.991474,-0.991474
4,Sylvia FABRE,2,4.0,2023-10-17 22:22:10+00:00,2023-10-17,MaBanqueBNPParibas,pas d imprimé prévu pour transferts de compte ...,imprimé prévu transferts compte compte demande...,imprimer prévoir transfert compte compte deman...,-1,0.982604,-0.982604


### Remettre en datetime

In [ ]:
### Retirer les minutes / secondes / décalage
df["date_avis"] = df["date_avis"].str.split(" ").str[0]
df["date_experience"] = df["date_experience"].str.split(" ").str[0]

### Transformation en datetime
df["date_avis"] = pd.to_datetime(df["date_avis"], format='%Y-%m-%d')
df["date_experience"] = pd.to_datetime(df["date_experience"], format='%Y-%m-%d')

df.info()

### Ajouter les colonnes jour / mois / heure / année et les catégoriser + rajouter longueur des textes

In [ ]:
df.text_total[0]

In [ ]:
### Ajout des colonnes jour / mois / heure / année pour avis

df["jour_avis"] = df["date_avis"].dt.day
df["mois_avis"] = df["date_avis"].dt.month
df["heure_avis"] = df["date_avis"].dt.hour
df["annee_avis"] = df["date_avis"].dt.year

df["jour_experience"] = df["date_experience"].dt.day
df["mois_experience"] = df["date_experience"].dt.month
df["heure_experience"] = df["date_experience"].dt.hour
df["annee_experience"] = df["date_experience"].dt.year

### Catégoriser en jour et nuit

df["periode_avis"] = df["heure_avis"].apply(lambda x: "jour" if 6 <= x <= 22 else "nuit")
df["longueur_texte"] = df["text_avis"].apply(len)

df.head()

In [ ]:
import spacy
batch_size = 1000  # Adjust the batch size based on your system

# Function to remove stopwords
def remove_stopwords(doc):
    return " ".join([token.text for token in doc if not token.is_stop])

# Process texts in batches for stopword removal
processed_texts = []
i = 0
for doc in nlp.pipe(df.text_total.iloc[0:10000], batch_size=batch_size):
    i += 1
    processed_text = remove_stopwords(doc)
    processed_texts.append(processed_text)
    print(f'batch {i} sur {len(df)/batch_size}')

In [5]:
# enlever les stopwords: essai pipe spacy
"""
import spacy 
df['text_total'] = df['text_total'].fillna('').astype(str)
df['text_total'] = [message.lower() for message in df['text_total']]
nlp = spacy.load("../models/fr_core_news_md-3.7.0-py3-none-any/fr_core_news_md/fr_core_news_md-3.7.0")
stop_words = spacy.lang.fr.stop_words.STOP_WORDS
"""

In [8]:
df.text_stop[0]

'orgueil gestionnaire mauvaise expérience , gestionnaire regard haut , demande client , choquant , s agit dame malezieu vélizy - villacoublay , j jamais vie , croit trop parfaite ? ? ? ? peau matte pourtant , arrive croire , métier crois , contact clientèle , réaction vaut 0 , passe message supérieur'

In [6]:
def stop_stop(review):
    filtre = []
    texte = nlp(review)
    liste_texte = list(texte)
    for word in liste_texte:
        if str(word) not in stop_words:
            filtre.append(str(word))
    return(" ".join(filtre))

liste_filtrée =[]
round_count = 0
for review in df.text_total:
    if round_count % 10000 == 0:
        print("passage de l'avis numéro :", round_count)
    filt_review = stop_stop(review)
    liste_filtrée.append(filt_review)
    round_count = round_count + 1
df["text_stop"] = liste_filtrée

passage de l'avis numéro : 0
passage de l'avis numéro : 10000
passage de l'avis numéro : 20000
passage de l'avis numéro : 30000
passage de l'avis numéro : 40000
passage de l'avis numéro : 50000
passage de l'avis numéro : 60000
passage de l'avis numéro : 70000
passage de l'avis numéro : 80000
passage de l'avis numéro : 90000
passage de l'avis numéro : 100000


In [12]:
batch_size = 1000
liste_lemma = []
i = 0
for doc in nlp.pipe(df.text_stop, batch_size=batch_size):
    i +=1
    lemmas = [token.lemma_ for token in doc]
    liste_lemma.append(" ".join(lemmas))
    print(i ,"reviews passed")
df['text_lemma'] = liste_lemma

1 reviews passed
2 reviews passed
3 reviews passed
4 reviews passed
5 reviews passed
6 reviews passed
7 reviews passed
8 reviews passed
9 reviews passed
10 reviews passed
11 reviews passed
12 reviews passed
13 reviews passed
14 reviews passed
15 reviews passed
16 reviews passed
17 reviews passed
18 reviews passed
19 reviews passed
20 reviews passed
21 reviews passed
22 reviews passed
23 reviews passed
24 reviews passed
25 reviews passed
26 reviews passed
27 reviews passed
28 reviews passed
29 reviews passed
30 reviews passed
31 reviews passed
32 reviews passed
33 reviews passed
34 reviews passed
35 reviews passed
36 reviews passed
37 reviews passed
38 reviews passed
39 reviews passed
40 reviews passed
41 reviews passed
42 reviews passed
43 reviews passed
44 reviews passed
45 reviews passed
46 reviews passed
47 reviews passed
48 reviews passed
49 reviews passed
50 reviews passed
51 reviews passed
52 reviews passed
53 reviews passed
54 reviews passed
55 reviews passed
56 reviews passed
5

In [ ]:
df.to_csv("../data/avis/df_clean_noYC_lemma.csv")

In [ ]:
# ancienne version lemma sans batch
"""
def lemma(review):
    texte = nlp(review)
    temp_lemma = []
    for word in texte:
        temp_lemma.append(word.lemma_)
    return(" ".join(temp_lemma))


liste_lemma = []
round_count = 0
for review in df.text_stop:
    if round_count % 10000 == 0:
        print("passage de l'avis numéro :", round_count)
    liste_lemma.append(lemma(review))
    round_count = round_count + 1

df['text_lemma'] = liste_lemma
"""